In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [2]:
list=[[[0.27047928569511825,0.5312608102025099,0.19825990410237174]],
      [[0.06711381377029987,0.8775456658890036,0.05534052034069637]],
      [[0.10847074295048188,0.04602848157663474,0.8455007754728833]]]
df=spark.createDataFrame(list, ['probability'])
df.show(truncate=False)

+--------------------------------------------------------------+
probability |
+--------------------------------------------------------------+
[0.27047928569511825, 0.5312608102025099, 0.19825990410237174]|
[0.06711381377029987, 0.8775456658890036, 0.05534052034069637]|
[0.10847074295048188, 0.04602848157663474, 0.8455007754728833]|
+--------------------------------------------------------------+

In [3]:
df.withColumn("largest_1", F.array_max(F.col("probability")))\
  .withColumn("index_1", F.expr("""array_position(probability,largest_1)"""))\
  .withColumn("largest_2", F.expr("""array_max((filter(probability,x-> x!=largest_1)))"""))\
  .withColumn("index_2", F.expr("""array_position(probability, float(largest_2))""")).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1431059237576722> in <module> 
 1 df . withColumn ( "largest_1" , F . array_max ( F . col ( "probability" ) ) ) \ 
 2 . withColumn ( "index_1" , F . expr ( """array_position(probability,largest_1)""" ) ) \ 
 ----> 3 . withColumn ( "largest_2" , F . expr ( """array_max((filter(probability,x-> x!=largest_1)))""" ) ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 382 """
 383 if isinstance ( truncate , bool ) and truncate : 
 --> 384 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 385 else : 
 386 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o635.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 6, localhost, executor driver): java.lang.ClassCastException: java.lang.Double cannot be cast to java.lang.Float
	at scala.runtime.BoxesRunTime.unboxToFloat(BoxesRunTime.java:109)
	at org.apache.spark.sql.catalyst.util.GenericArrayData.getFloat(GenericArrayData.scala:72)
	at com.databricks.sql.expressions.codegen.RuntimeUtils.arrayPosition(RuntimeUtils.java:151)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_1$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:62)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:159)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:158)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:140)
	at org.apache.spark.scheduler.Task.run(Task.scala:113)
	at org.apache.spark.executor.Executor$TaskRunner$$anonfun$13.apply(Executor.scala:537)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1541)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:543)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2362)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2350)
	at org.apache.

In [4]:
+-------------------------------------------------------------------------------------------------------------- -----+
|probability                                                 |     largest_1    |index_1|largest_2          |index_2 |
+------------------------------------------------------------|------------------|-------|-------------------|--------+
|[0.27047928569511825,0.5312608102025099,0.19825990410237174]|0.5312608102025099|   1   |0.27047928569511825| 0      |
|[0.06711381377029987,0.8775456658890036,0.05534052034069637]|0.8775456658890036|   1   |0.06711381377029987| 0      |
|[0.10847074295048188,0.04602848157663474,0.8455007754728833]|0.8455007754728833|   2   |0.10847074295048188| 0      |
+--------------------------------------------------------------------------------------------------------------------+

In [5]:
df.withColumn("largest_1", F.array_max(F.col("probability")))\
  .withColumn("largest_2", F.expr("""array_position(probability,largest_1)"""))\
  .selectExpr("*","array_position(probability,largest_1) -1 index_1","array_position(probability,largest_2) -1 index_2").show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1431059237576723> in <module> 
 1 df . withColumn ( "largest_1" , F . array_max ( F . col ( "probability" ) ) ) \ 
 2 . withColumn ( "largest_2" , F . expr ( """array_position(probability,largest_1)""" ) ) \ 
 ----> 3 . selectExpr ( "*" , "array_position(probability,largest_1) -1 index_1" , "array_position(probability,largest_2) -1 index_2" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 382 """
 383 if isinstance ( truncate , bool ) and truncate : 
 --> 384 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 385 else : 
 386 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o959.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 10, localhost, executor driver): java.lang.ClassCastException: java.lang.Double cannot be cast to java.lang.Float
	at scala.runtime.BoxesRunTime.unboxToFloat(BoxesRunTime.java:109)
	at org.apache.spark.sql.catalyst.util.GenericArrayData.getFloat(GenericArrayData.scala:72)
	at com.databricks.sql.expressions.codegen.RuntimeUtils.arrayPosition(RuntimeUtils.java:151)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_1$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:62)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:159)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:158)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:140)
	at org.apache.spark.scheduler.Task.run(Task.scala:113)
	at org.apache.spark.executor.Executor$TaskRunner$$anonfun$13.apply(Executor.scala:537)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1541)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:543)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2362)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGSched

In [6]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
df.withColumn("largest_1", F.array_max(F.col("probability")))\
  .withColumn("stringlist", F.col("probability").cast(ArrayType(StringType())))\
  .withColumn("index_1", F.expr("""array_position(stringlist, string(largest_1))-1"""))\
  .withColumn("largest_2", F.expr("""array_max((filter(probability,x-> x!=largest_1)))"""))\
  .withColumn("index_2", F.expr("""array_position(stringlist, string(largest_2))-1"""))\
  .drop("stringlist")\
  .printSchema()

root
-- probability: array (nullable = true)
 |-- element: double (containsNull = true)
-- largest_1: double (nullable = true)
-- index_1: long (nullable = true)
-- largest_2: double (nullable = true)
-- index_2: long (nullable = true)

In [7]:
from pyspark.sql.functions import *
df.withColumn("sort_arr",array_sort(col("probability"))).\
withColumn("largest_1",element_at(col("sort_arr"),-1)).\
withColumn("largest_2",element_at(col("sort_arr"),-2)).\
selectExpr("*","array_position(probability,largest_1) -1 index_1","array_position(probability,largest_2) -1 index_2").\
drop("sort_arr").\
show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1431059237576725> in <module> 
 4 withColumn ( "largest_2" , element_at ( col ( "sort_arr" ) , - 2 ) . cast ( "float" ) ) . \ 
 5 selectExpr ( "*" , "array_position(probability,largest_1) -1 index_1" , "array_position(probability,largest_2) -1 index_2" ) . \ 
 ----> 6 drop ( "sort_arr" ) . \ 
 7 show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 382 """
 383 if isinstance ( truncate , bool ) and truncate : 
 --> 384 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 385 else : 
 386 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o2048.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 35.0 failed 1 times, most recent failure: Lost task 0.0 in stage 35.0 (TID 35, localhost, executor driver): java.lang.ClassCastException: java.lang.Double cannot be cast to java.lang.Float
	at scala.runtime.BoxesRunTime.unboxToFloat(BoxesRunTime.java:109)
	at org.apache.spark.sql.catalyst.util.GenericArrayData.getFloat(GenericArrayData.scala:72)
	at com.databricks.sql.expressions.codegen.RuntimeUtils.arrayPosition(RuntimeUtils.java:151)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_1$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:62)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:159)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:158)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:140)
	at org.apache.spark.scheduler.Task.run(Task.scala:113)
	at org.apache.spark.executor.Executor$TaskRunner$$anonfun$13.apply(Executor.scala:537)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1541)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:543)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2362)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2350)
	at org.apache.spark.scheduler.D

In [8]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

sdf1 = spark.createDataFrame([
    ("123", "A", [1, 2, 3]),
    ("123","B", [4, 5]),
    ("456","C", [1, 2]),
    ("456","D", [3, 4, 5]),
], ["id1", "name", "resources"])

sdf2 = spark.createDataFrame([
    ("123", 1, "R1"),
    ("123", 2, "R2"),
    ("123", 3, "R3"),
    ("123", 4, "R4"),
    ("123", 5, "R5"),
    ("456", 1, "R1"),
    ("456", 2, "R2"),
    ("456", 3, "R7"),
    ("456", 4, "R8"),
    ("456", 5, "R9")
], ["id2", "resource_id", "name"])

sdf1.show()
sdf2.show()

+---+----+---------+
id1|name|resources|
+---+----+---------+
123| A|[1, 2, 3]|
123| B| [4, 5]|
456| C| [1, 2]|
456| D|[3, 4, 5]|
+---+----+---------+

+---+-----------+----+
id2|resource_id|name|
+---+-----------+----+
123| 1| R1|
123| 2| R2|
123| 3| R3|
123| 4| R4|
123| 5| R5|
456| 1| R1|
456| 2| R2|
456| 3| R7|
456| 4| R8|
456| 5| R9|
+---+-----------+----+

In [9]:

res_sdf = sdf1.join(sdf2, on=[(sdf1.id1 == sdf2.id2) & F.expr("""array_contains(resources, resource_id)""")], how='left')
res_sdf.groupBy(sdf1.id1,sdf1.name,"resources").agg(F.collect_list(sdf2.name).alias("New Column")).orderBy("id1").show()

+---+----+---------+------------+
id1|name|resources| New Column|
+---+----+---------+------------+
123| A|[1, 2, 3]|[R1, R2, R3]|
123| B| [4, 5]| [R4, R5]|
456| C| [1, 2]| [R1, R2]|
456| D|[3, 4, 5]|[R7, R8, R9]|
+---+----+---------+------------+

In [10]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

sdf2= sdf2.withColumnRenamed("name","name1")
res_sdf = sdf1.join(sdf2, on=[(sdf1.id1 == sdf2.id2) & F.expr("""array_contains(resources, resource_id)""")], how='left')
w1=Window().partitionBy("id1","resources").orderBy("name1")
w2=Window().partitionBy("id1","resources").orderBy("name1").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
res_sdf.withColumn("New Column", F.collect_list("name1").over(w1)).drop("id2","resource_id")\
       .withColumn("last", F.last("New Column").over(w2))\
       .filter(F.col("New Column")==F.col("last"))\
       .drop("name1","last")\
       .orderBy("name")\
       .show()

+---+----+---------+------------+
id1|name|resources| New Column|
+---+----+---------+------------+
123| A|[1, 2, 3]|[R1, R2, R3]|
123| B| [4, 5]| [R4, R5]|
456| C| [1, 2]| [R1, R2]|
456| D|[3, 4, 5]|[R7, R8, R9]|
+---+----+---------+------------+

In [11]:
res_sdf = sdf1.join(sdf2, on=[(sdf1.id1 == sdf2.id2) & F.array_contains(sdf1.resources, sdf2.resource_id)], how='left')

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-493714868398103> in <module> 
 ----> 1 res_sdf = sdf1 . join ( sdf2 , on = [ ( sdf1 . id1 == sdf2 . id2 ) & F . array_contains ( sdf1 . resources , sdf2 . resource_id ) ] , how = 'left' ) 

 /databricks/spark/python/pyspark/sql/functions.py in array_contains (col, value) 
 1886 """
 1887 sc = SparkContext . _active_spark_context
 -> 1888 return Column ( sc . _jvm . functions . array_contains ( _to_java_column ( col ) , value ) ) 
 1889 
 1890 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1246 
 1247 def __call__ ( self , * args ) : 
 -> 1248 args_command , temp_args = self . _build_args ( * args ) 
 1249 
 1250 command = proto . CALL_COMMAND_NAME + \ 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _build_args (self, *args) 
 1210 def _build_args ( self , * args ) : 
 1211 if self . converters is not None and len ( self . converters ) > 0 : 
 -> 1212 ( new_args , temp_args ) = self . _get_args ( args ) 
 1213 else : 
 1214 new_args = args

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _get_args (self, args) 
 1197 for converter in self . gateway_client . converters : 
 1198 if converter . can_convert ( arg ) : 
 -> 1199 temp_arg = converter . convert ( arg , self . gateway_client ) 
 1200 temp_args . append ( temp_arg ) 
 1201 new_args . append ( temp_arg ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_collections.py in convert (self, object, gateway_client) 
 498 ArrayList = JavaClass ( "java.util.ArrayList" , gateway_client ) 
 499 java_list = ArrayList ( ) 
 --> 500 for element in object : 
 501 java_list . add ( element ) 
 502 return java_list

 /databricks/spark/python/pyspark/sql/column.py in __iter__ (self) 
 342 
 343 def __iter__ ( self ) : 
 --> 344 raise TypeError ( "Column is not iterable" ) 
 345 
 346 # string methods 

 TypeError : Column is not iterable

In [12]:
+----+-----+-----------+-------------+
|id1 |name |resources  |New Column   |
+----+-----+-----------+-------------+
|123 |A    |[1, 2, 3]  |[R1, R2, R3] |
|123 |B    |[4, 5]     |[R4, R5]     |
|456 |C    |[1, 2]     |[R1, R2]     |
|456 |D    |[3, 4, 5]  |[R7, R8, R9] |
+----+---------+------+--------------+ 